#### Sequential Learning App for Materials Discovery - *SLAMD*

In [440]:
#Imports

from resample.jackknife import resample as b_resample
from numpy import random
import ipywidgets as widgets
from ipywidgets import RadioButtons,FloatRangeSlider,Button,IntSlider
from IPython.display import display,Markdown
import pandas as pd
import sys
import statistics
from io import StringIO
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.spatial import distance_matrix

#import imsl.data_mining as dm

import time
import seaborn as sns

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor as SKRFR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

from sklearn import preprocessing

#FUELS-imports
from lolopy.learners import RandomForestRegressor
from operator import add
#from sklearn.model_selection import train_test_split

from sklearn.manifold import TSNE
import seaborn as sns

from matplotlib.colors import Colormap

In [441]:
#Components

tab = widgets.Tab()    

out = widgets.Output(layout={'border': '1px solid black'})
out_plotting= widgets.Output(layout={'border': '1px solid black'})



out_algo=widgets.Output(layout={'border': '1px solid black'})
out_algo_alt=widgets.Output(layout={'border': '1px solid black'})


out_perform_experiment=widgets.Output(layout={'border': '1px solid black'})
out_input_space=widgets.Output(layout={'border': '1px solid black'})

out_res=widgets.Output(layout={'border': '1px solid black'})


up = widgets.FileUpload(accept="", multiple=False)

delim = widgets.RadioButtons(
    options=[';', ','],
    description='Separator: ',
    disabled=False)

delim_dec = widgets.RadioButtons(
    options=[',', '.'],
    description='Decimal Delim: ',
    disabled=False)

eraser = widgets.SelectMultiple(
    options=['tab','"',"%"],
    value=['tab'],
    #rows=10,
    description='Eraser: ',
    disabled=False)

rows = widgets.IntSlider(
    value=0,
    step=1,
    description='# of lines:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

##ML-###pre processing

feature_selector=widgets.SelectMultiple(
    options=[],
    #tooltip("Select the Input Features")
    #rows=10,
    description='Features',
    disabled=False,
layout=widgets.Layout(width='50%'))
    
target_selection=widgets.SelectMultiple(
    options=[],
    #value='',
    placeholder='select the Target Variable',
    description='Target-Variable',
    disabled=False,
layout=widgets.Layout(width='50%'))

selector_plot_variable=widgets.SelectMultiple(
    options=[],
    #tooltip("Select the Input Features")
    #rows=10,
    description='Features',
    disabled=False,
layout=widgets.Layout(width='50%'))
    





graph_type = widgets.Dropdown(
    options=['Choose the Graph Type','Scatter', 'Scatter Matrix', 'Correlation Heatmap'],
    value='Choose the Graph Type',
    description='Chart Type:',
    disabled=False,
    layout=widgets.Layout(width='50%'))

x_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='X-Axis:',
    disabled=False)
y_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='Y-Axis:',
    disabled=False)
color_picker = widgets.ColorPicker(
    concise=False,
    description='Color Picker: ',
    value='lightblue',
    disabled=False)
toggle = widgets.ToggleButtons(
    options=['Preview  ', 'Info  ', 'Stats  '],
    description='Options',
    disabled=False,
    button_style='warning',
    icons=['search', 'info', 'tachometer'])

select_x=widgets.Dropdown(
    options=[''],
    value='',
    placeholder='select X-Axis',
    description='X-Axis:',
    disabled=False,
layout=widgets.Layout(width='50%'))
select_y=widgets.Dropdown(
    options=[''],
    value='',
    placeholder='select Y-Axis',
    description='Y-Axis:',
    disabled=False,
layout=widgets.Layout(width='50%'))

select_hue=widgets.Dropdown(
    options=[''],
    value='',
    placeholder='select the hue',
    description='Hue:',
    disabled=False,
layout=widgets.Layout(width='50%'))

select_size=widgets.Dropdown(
    options=[''],
    value='',
    placeholder='select the Size',
    description='Size:',
    disabled=False,
layout=widgets.Layout(width='50%'))


select_strategy=widgets.Dropdown(
    options=['MEI (exploit)','MU (explore)','MLI (explore & exploit)','MEID (exploit)','MLID (explore & exploit)'],
    value='MEI (exploit)',
    placeholder='select the strategy',
    description='Strategy:',
    disabled=False,
layout=widgets.Layout(width='50%'))

select_model=widgets.Dropdown(
    options=['lolo Random Forrest (RF) - quick (requ. min 8 init. samples)','Decision Trees (DT) - quick','Bagged Tree Ensembles (TE) - slow','Gaussian Process Regression (GPR) - quick'],
    value='Decision Trees (DT) - quick',
    placeholder='select the Model',
    description='Model:',
    disabled=False,
layout=widgets.Layout(width='50%'))




button_confirm_strategy=widgets.Button(
    description='Confirm Strategy ',
    disabled=False,
    button_style='success',
    tooltip='Confirm selected Strategy',
    icon='check',
layout=widgets.Layout(width='50%',height ='inherit'))




#Options
box_features=widgets.VBox([])




#container_feature_options=widgets.VBox([widgets.HBox([box_features,box_features_slider]),box_confirm])
#feature_options= widgets.Accordion(children=[container_feature_options])

#feature_options.set_title(0,"Specify your Target")

In [442]:
#Buttons Anfang
button_upload = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='success',
    tooltip='Click to Upload',
    icon='check',
layout=widgets.Layout(width='50%'))

button_preview = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search',
layout=widgets.Layout(width='50%'))



#Plotting
button_plot = widgets.Button(
    description='Plot',
    disabled=False,
    button_style='warning',
    tooltip='Click to Plot',
    icon='pencil',
layout=widgets.Layout(width='100%'))




###Algos
button_confirm_feature=widgets.Button(
    description='Select Features ',
    disabled=False,
    button_style='success',
    tooltip='Confirm Feature Selection of Training Features',
    icon='check',
layout=widgets.Layout(width='50%',height ='inherit'))

button_confirm_target=widgets.Button(
    description='Select Target',
    disabled=False,
    button_style='success',
    tooltip='Confirm the selected Target Variable',
    icon='check',
layout=widgets.Layout(width='50%',height ='inherit'))

button_confirm_plot_var=widgets.Button(
    description='Confirm Selection',
    disabled=False,
    button_style='success',
    tooltip='Confirm the selected Target Variable',
    icon='check',
    layout=widgets.Layout(width='50%',height ='inherit'))

button_plot_comparision=widgets.Button(
    description='Compare',
    disabled=False,
    button_style='warning',
    tooltip='Simplify the Columns',
    icon='fa-bar-chart',
    layout=widgets.Layout(width='50%')
)


button_confirm_options=Button(
    description='Confirm Options ',
    disabled=False,
    button_style='success',
    tooltip='Confirm options',
    icon='check',
    layout=widgets.Layout(width='50%',height ='inherit'))
                           
###Algos ende
##ML End

button_custom_strategey=widgets.Button(
    description='Customize',
    disabled=False,
    button_style='warning',
    tooltip='Simplify the Columns',
    icon='fa-bar-chart',
    layout=widgets.Layout(width='50%')
       
    
)
button_perform_experiment=widgets.Button(
    description='Run',
    disabled=False,
    button_style='warning',
    tooltip='Perform Experiment',
    icon='fa-calculator',
    layout=widgets.Layout(width='100%'))


button_cancel_experiment=widgets.Button(
    description='Stop',
    disabled=False,
    button_style='danger',
    tooltip='Stop Experiment',
    icon='fa-stop',
    layout=widgets.Layout(width='100%'))
    
#Buttons Ende

In [443]:
#FileUpload Tab
accordion = widgets.Accordion(children=[
    up, 
    widgets.HBox([delim, delim_dec, eraser]), 
    rows])

accordion.set_title(0, 'File Selection')
accordion.set_title(1, 'Delimiter')
accordion.set_title(2, 'Skip Rows')


accordion_box = widgets.VBox([
    accordion, 
    widgets.HBox([button_preview, button_upload ]),
    out
])

In [444]:
#Sequential Learning Tab 
slider_of_for_dist=widgets.FloatSlider(
    value=90,
    min=0,
    max=100,
    step=1,
    description='Prediction threshold for considered samples distances (Quantile):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    
    layout=widgets.Layout(width='50%')
)

quantile_tar_slider= widgets.FloatSlider(
    value=95,
    min=1,
    max=100,
    step=1,
    description='Target threshold (Quantile):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    
    layout=widgets.Layout(width='50%')
)


quantile_sample_slider= widgets.FloatSlider(
    value=50,
    min=1,
    max=100,
    step=5,
    description='Sample threshold (Quantile):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    
    layout=widgets.Layout(width='50%')
)
initial_sample_size_text=widgets.BoundedIntText(
    value=4,
    min=2,
    max=20,
    step=1,
    description='Initial Sample Size:',
    disabled=False,
    layout=widgets.Layout(width='50%')
)

#slider_quantile_container=widgets.VBox([widgets.HBox(children=[quantile_tar_slider,quantile_sample_slider,initial_sample_size_text])])

box_features_slider=widgets.VBox([])


DataPre_sl=widgets.VBox([
        widgets.HBox([feature_selector, button_confirm_feature ]),
        widgets.HBox([target_selection, button_confirm_target]),
#      slider_quantile_container,
#feature_options,
#button_split_data,
    box_features,
    quantile_tar_slider,
    quantile_sample_slider,
    initial_sample_size_text,
])

DataPre=widgets.VBox([
        widgets.HBox([feature_selector, button_confirm_feature ]),
        widgets.HBox([target_selection, button_confirm_target]),
       
    
])

iterations=widgets.IntSlider(
    value=30,
    min=1,
    max=100,
    step=1,
    description='# of SL runs:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='50%')
)
show_results_button= widgets.Button(
    description='Show Results',
    disabled=False,
    button_style='warning',
    tooltip='Show Results',
    icon='fa-bar-chart',
    layout=widgets.Layout(width='100%')
)

custom_container=widgets.VBox([widgets.HBox([])])
results_container=widgets.VBox([widgets.HBox([])])
strategy_container=widgets.HBox([select_strategy, button_confirm_strategy ])

start_and_stop_sl_container=widgets.HBox([button_perform_experiment])

sl_settings= widgets.VBox([
    strategy_container,
    custom_container,
    widgets.HBox([select_model,iterations]),
    start_and_stop_sl_container,
    out_perform_experiment,
    out_input_space
])



sl_accordion=widgets.Accordion(children=[DataPre_sl,sl_settings])
sl_accordion.set_title(0,"Settings")
sl_accordion.set_title(1,"Sequential Learning Parameters")

In [445]:
#Plotting Tab



container_plot_options= widgets.VBox([])
button_container=widgets.HBox([button_plot])

plotting=widgets.VBox(children=[widgets.VBox( [
        widgets.HBox([graph_type]),
        container_plot_options,
        button_container,
        out_plotting
        
        
]


)])

children = [
    accordion_box, 
    widgets.VBox([toggle, out]),
    plotting,
    #plot_accordion,
    sl_accordion
    
   ]

In [446]:
#Tabs
tab.children = children
tab.set_title(0, "Upload")
tab.set_title(1, "Data Info")
tab.set_title(2, "Design Space Explorer")
tab.set_title(3, "Sequential Learning")

In [447]:
#EventHandling

def content_parser():
    if up.value == {}:
        """with out:
            out.clear_output
            display(Markdown('No CSV loaded'))
            #print('No CSV loaded')    """
    else:
        typ, content = "", ""
        up_value = up.value
        for i in up_value.keys():
            typ = up_value[i]["metadata"]["type"]
            if typ == "text/csv":
                content = up_value[i]["content"]
                content_str = str(content, 'utf-8')

                if eraser.value != {}: 
                    for val in eraser.value:
                        if val == "tab":
                            content_str = content_str.replace("\t","")
                        elif val =="%":
                            content_str = content_str.replace("\t","")
                        else:
                            content_str = content_str.replace(val,"")
                if content_str != "":
                    str_io = StringIO(content_str) 
                    return str_io
def df_converter():
    content = content_parser()
    if content is not None:
            df = pd.read_csv(content, sep=delim.value, index_col=False, skiprows=rows.value,decimal=delim_dec.value)
            df=df.apply(pd.to_numeric,errors="coerce")
            
            
            
                
            return df
    else:
        return None
def preview():
    
    df = df_converter()
    with out:
        out.clear_output()
        display(Markdown('Now this is how your DF looks like:'))
        
        if df is not None:
            print(df.head(10))
        else:
            display(Markdown('Configuration is wrong/missing...'))
            
def upload():
    
    df = df_converter()
    with out:
        out.clear_output()
        display(Markdown('Your uploaded DF looks like:'))
       
        if df is not None:
            print(df)
            x_axis.options = df.columns
            y_axis.options = df.columns
            feature_selector.options= df.columns
            select_x.options=df.columns
            select_y.options=df.columns
            select_size.options=df.columns
            select_hue.options=df.columns
            selector_plot_variable.options=df.columns
            
        else:
            display(Markdown('Configuration is wrong/missing...'))
            #print('Configuration is wrong/missing...')
def desc():
    info_level = toggle.value
    if info_level != {}:
        df = df_converter()
        with out:
            out.clear_output()
            print('\n ------ Data {} ------ \n'.format(
                info_level))
            if df is not None:
                if info_level == 'Info  ':
                    print(df.info(verbose=True))
                elif info_level == 'Stats  ':
                    print(df.describe())
                elif info_level == 'Preview  ':
                    print(df.head(5))
                else:
                    display(Markdown('Configuration is wrong/missing...'))
       



    
def plot():
    graph = graph_type.value
    if graph=="Scatter":
        plot_scatter()
    elif graph=="Correlation Heatmap":
            plot_heat()
    elif graph=="Scatter Matrix":
            plot_pairwise()
          

        
def plot_pairwise():
    
    df =confirm_var()
    
    with out_plotting:
        out_plotting.clear_output()
        sns.pairplot(df)
        plt.show()

def plot_heat():
    
    df = confirm_var()
    
    with out_plotting:
        out_plotting.clear_output()
        corr = df.corr()

        plt.figure(figsize=(9,7))
        sns.heatmap(corr, annot=True, cmap='Blues')
        b, t = plt.ylim()
        plt.ylim(b+0.5, t-0.5)
        plt.title("Feature Correlation Heatmap")
        plt.show()
            
def plot_scatter():
    data=df_converter()
    with out_plotting:
        out_plotting.clear_output()
        fig, ax = plt.subplots(figsize=(10,7))
        #not generic
        sns.scatterplot(y=select_y.value, x=select_x.value, hue=select_hue.value, size=select_size.value, data=data, ax=ax, sizes=(50, 300))
        ax.set_title(select_y.value+ "vs"+ select_x.value)
        ax.legend(loc="upper left", bbox_to_anchor=(1,1))
        plt.show()
        plt.close(fig)      

In [448]:
#ml eventhandling
#pre pocessing

def confirm_options():
    items=box_features_slider.children
    df = df_converter()
    Y = df.loc[:,df.columns.isin(target_selection.value)]
    
    for slider in items:
            unt_grenz= slider.value[0]/100
            ob_grenz= slider.value[1]/100
            Y = Y[(Y >= Y.quantile(unt_grenz) ) & (Y <= Y.quantile(ob_grenz))]
            Y= Y.dropna()
    
    return Y   

#returns index of features
def confirm_features():
    df = df_converter()
    train = feature_selector.value
    target_selection.options=df.columns[~df.columns.isin(feature_selector.value)]
    train = df.columns[df.columns.isin(feature_selector.value)]
    return train
    
def confirm_var():
    df= df_converter()
    #df.columns[df.columns.isin(feature_selector.value)]
    #var = df.columns[df.columns.isin(selector_plot_variable.value)]
    selection = list(selector_plot_variable.value)
    var = df[selection]
 
    return var

    
#returns index of target    
def confirm_target():
    df = df_converter()
    #target_Var=target_selection.value
    #feature_selector.options= get_features()
    target = df.columns[df.columns.isin(target_selection.value)]
    
    
    if(len(box_features.children) == 0):
        create_dynamically_checkboxes()
   
    
    return target 
    
    
def create_slider_for_dist_quantile():
    strategy_container.children=[select_strategy,slider_of_for_dist,  button_confirm_strategy]
    
def create_dynamically_checkboxes():
    target= target_selection.value
    items = [RadioButtons(
    options=['maximize', 'minimize'],
    value='maximize', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description=feature,
    disabled=False
) for feature in target]
    box_features.children=items

In [449]:
class sequential_learning:
    
    xlabel="Sequential Learning Iteration"
    dataframe = df_converter()
    init_sample_size=4
    number_of_executions=0
    features_df=df_converter()
    target_df=df_converter()
    targ_q=quantile_tar_slider.value
    samp_q=quantile_sample_slider.value
    min_distances_list=[]
    
    tries_list=np.empty(iterations.value)
    
    tries_list_rand_pick=np.empty(iterations.value)
    
    
    y_pred_dtr_mean=None
    y_pred_dtr_std=None
    y_pred_dtr=None
    SampIdx=None
    PredIdx=None
        
    def __init__(self,dataframe):  #constructor
        self.dataframe= dataframe
        
    def apply_feature_selection_to_df(self,dataframe):
        self.features_df = self.dataframe[confirm_features()]    
    
    def apply_target_selection_to_df(self,dataframe):
        self.target_df= self.dataframe[confirm_target()]     
        
    def standardize_data(self):
        dataframe_norm=(self.dataframe-self.dataframe.mean())/self.dataframe.std()
        target_df_norm=(self.target_df-self.target_df.mean())/self.target_df.std()
        features_df_norm=(self.features_df-self.features_df.mean())/self.features_df.std()
        self.features_df=features_df_norm
        self.target_df=target_df_norm
        self.dataframe=dataframe_norm
        return self.features_df, self.target_df, self.dataframe
        
    
    def set_init_sample_size(self):
        self.init_sample_size=initial_sample_size_text.value
    
        
    def set_number_of_ex(self):
        self.number_of_executions= iterations.value
        
        
        
        
    def init_sampling(self):
        self.samp_q= quantile_sample_slider.value/100
        chosen_columns = confirm_target()
        
        
        #sum dataframe
        sum_ = self.dataframe[chosen_columns].sum(axis=1).to_frame()
        
        #sample quantile to restrict initial sampling 
        samp_q_t=sum_.quantile(self.samp_q)
        
        Index_label=np.where(sum_ < samp_q_t )
        Index_label=Index_label[0]
        
        init_sample_set = np.ones((0,self.init_sample_size))
        
        
        for i in range(self.number_of_executions):
                    
                    init_sample_set=np.vstack([init_sample_set, random.choice(Index_label,self.init_sample_size)])
       
        return init_sample_set
    
    def decide_max_or_min(self):
        
        max_or_min_optimizations_list=[]
        
        for decide in range(len(self.target_df.columns)):
            
                            
            max_or_min_optimizations_list.append(box_features.children[decide].value)
        
        return max_or_min_optimizations_list
            
    def start_sequential_learning(self):
        self.tries_list=np.empty(iterations.value)
        self.tries_list.fill(np.nan)
        self.tries_list_rand_pick=np.ones(iterations.value)

        
        distances=[]
        current_distances_list=[0]    
        with out_perform_experiment:
                display(Markdown('Sequential Learning is running...'))
       
        
        init_sample_set=self.init_sampling()
       
        
        for i in range(self.number_of_executions):
            self.perform_random_pick(i)
            
            current_distances_list=[0]
            self.SampIdx=init_sample_set[i].astype(int)
            self.PredIdx=self.dataframe
            self.PredIdx = self.PredIdx.drop(self.PredIdx.index[self.SampIdx]).index
            #init train
            self.decide_model(select_model.value)
            self.tries_list[i]=self.init_sample_size
            decide_optimization=self.decide_max_or_min()
            
            for k in decide_optimization:
                index_of_target = decide_optimization.index(k)
                
                if (k == "minimize"): 
                    self.target_df.iloc[:,index_of_target]*(-1)
                    
                    
            #set quantile
            self.targ_q = quantile_tar_slider.value/100
            sum_ = self.dataframe[confirm_target()].sum(axis=1).to_frame()
            targ_q_t= sum_.quantile(self.targ_q)
            
            
            while sum_.loc[self.SampIdx].max().values <= targ_q_t.values:
                            
                            #Update_Idx_with_Strategy 
                            self.update_strategy(confirm_strategy())

                            #Train Model
                            self.decide_model(select_model.value)
                            
                            schwellwert=sum_.quantile(self.targ_q)
                            Index_c=np.where(sum_ >= schwellwert )
                            Index_c=Index_c[0]

                            distance= distance_matrix(self.dataframe.loc[self.SampIdx],self.dataframe.iloc[Index_c])
                            distance=distance.min()
                            current_distances_list.append(distance)
                            
                            self.tries_list[i]=self.tries_list[i]+1   
                        
                        
            
            
            
            distances.append(current_distances_list)
        
                
        
        ##Distance Plot
        
            with out_perform_experiment:
                fig1 = plt.figure(figsize=(10, 6))
                ax3 = plt.axes()
                ax3.set_title('Distance to target for each SL run')
                plt.xlabel('Number of SL iterations')
                plt.ylabel("Distance to target")

                for runs in range(len(distances)):
                    ax3.plot(distances[runs], color='k')
                
                #plt.close(fig1)
            with out_perform_experiment:
                out_perform_experiment.clear_output(wait=True)
                time.sleep(1.0)
                fig2=plt.figure(figsize=(10, 6))
                plt.xlabel('Number of required Experiments')
                plt.ylabel("Frequency")
                plt.title("Performance histogram for %s with strategy %s -  required experiments"%(select_model.value,select_strategy.value))
                #plt.hist([self.tries_list,self.tries_list_rand_pick],bins=len(self.tries_list),label=['SL Tries', 'Random Pick Tries'])         
                plt.hist([self.tries_list,self.tries_list_rand_pick],range=(1, len(self.features_df)),label=['SL', 'Random Process'])         
                plt.legend()

                plt.show()
                #plt.close(fig2)
            
                        

           
        with out_perform_experiment:
            display(Markdown('Sequential Learning performed ✅'))
            print('\033[1m' + 'Performance SL vs. RP:')
            print('\033[0m' + "requ. experiments SL (mean)",np.mean(self.tries_list))
            print("requ. experiments SL (90%)",np.quantile(self.tries_list,0.90))
            print("requ. experiments RP (mean)",np.mean(self.tries_list_rand_pick))
            print("requ. experiments RP (90%)",np.quantile(self.tries_list_rand_pick,0.90))
            print("SL - complete list of requ. experiments in each iteration",self.tries_list)
            print("RP - complete list of requ. experiments in each iteration",self.tries_list_rand_pick)
        self.plot_TSNE_input_space()        
    
    
    
    def perform_random_pick(self,acutal_iter):
        df_rand = self.dataframe.sample()
        self.targ_q = quantile_tar_slider.value/100
        sum_ = self.dataframe[confirm_target()].sum(axis=1).to_frame()
        targ_q_t= sum_.quantile(self.targ_q)
        rand_sum=sum_.sample(frac=1).to_numpy()
        run=0
        df_rand=-10000
        while df_rand <= targ_q_t.values:
            df_rand = rand_sum[run]
            self.tries_list_rand_pick[acutal_iter]=self.tries_list_rand_pick[acutal_iter]+1 
            run=run+1
        
    def decide_model(self,model):
        if model== 'lolo Random Forrest (RF) - quick (requ. min 8 init. samples)':
                    Expected_Pred, Uncertainty=  self.fit_RF_wJK()
        elif model == 'Decision Trees (DT) - quick':
                    Expected_Pred, Uncertainty =  self.fit_DT_wJK()
        elif model == 'Bagged Tree Ensembles (TE) - slow':
                    Expected_Pred, Uncertainty =  self.fit_TE_wJK()
        elif model == 'Gaussian Process Regression (GPR) - quick':
                    Expected_Pred, Uncertainty =  self.fit_GP()
                    
            
    def update_strategy(self, strategy):
        if strategy=='MEI (exploit)':
            self.updateIndexMEI()
        elif strategy=='MU (explore)':
            self.updateIndexMU()
        elif strategy=='MLI (explore & exploit)':
            self.updateIndexMLI()
        elif strategy=='MEID (exploit)':
            self.updateIndexMEID()        
        elif strategy=='MLID (explore & exploit)':
            self.updateIndexMLID()
        
    
    def updateIndexMEI(self):
            
            index_max = np.argmax(self.Expected_Pred)
            new_SampIdx=np.append(self.SampIdx,self.PredIdx[index_max])
            self.SampIdx=new_SampIdx
            new_PredIdx = np.delete(self.PredIdx, index_max)
            self.PredIdx=new_PredIdx
            
            
    def updateIndexMEID(self):
            schwellwert=np.quantile(self.Expected_Pred,slider_of_for_dist.value/100)
            Index_=np.where(self.Expected_Pred>=schwellwert )
            Index_=Index_[0]
            distance= distance_matrix(self.dataframe.loc[self.SampIdx],self.dataframe.iloc[Index_])
            min_distances=distance.min(1)
            #min_distances=np.mean(distance,axis=1)
            result = np.where(distance == min_distances.max())
            
            # zip the 2 arrays to get the exact coordinates
            listOfCordinates = list(zip(result[0], result[1]))    
            index_max=Index_[result[1]]
            new_SampIdx=np.append(self.SampIdx,self.PredIdx[index_max])
            self.SampIdx=new_SampIdx
            new_PredIdx = np.delete(self.PredIdx, index_max)
            self.PredIdx=new_PredIdx

    def updateIndexMLID(self):
        
            schwellwert=np.quantile((self.Expected_Pred.squeeze()+1.96*self.Uncertainty.squeeze()),slider_of_for_dist.value/100)
            Index_=np.where(self.Expected_Pred.squeeze()+1.96*self.Uncertainty.squeeze()>=schwellwert )
            Index_=Index_[0]
            distance= distance_matrix(self.dataframe.loc[self.SampIdx],self.dataframe.iloc[Index_])
            min_distances=distance.min(1)
            result = np.where(distance == min_distances.max())
            # zip the 2 arrays to get the exact coordinates
            listOfCordinates = list(zip(result[0], result[1]))    
            index_max=Index_[result[1]]
            new_SampIdx=np.append(self.SampIdx,self.PredIdx[index_max])
            self.SampIdx=new_SampIdx
            new_PredIdx = np.delete(self.PredIdx, index_max)
            self.PredIdx=new_PredIdx            
            
            
    def updateIndexMU(self):
            index_max = np.argmax(self.Uncertainty)
            new_SampIdx=np.append(self.SampIdx,self.PredIdx[index_max])
            self.SampIdx=new_SampIdx
            new_PredIdx = np.delete(self.PredIdx, index_max)
            self.PredIdx=new_PredIdx
            
            
    def updateIndexMLI(self):
            
            index_max = np.argmax(self.Expected_Pred.squeeze()+1.96*self.Uncertainty.squeeze())
            new_SampIdx=np.append(self.SampIdx,self.PredIdx[index_max])
            self.SampIdx=new_SampIdx
            new_PredIdx = np.delete(self.PredIdx, index_max)
            self.PredIdx=new_PredIdx
                          
    def plot_TSNE_input_space(self):
        
        
            X=self.features_df
            y=self.target_df

            tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
            tsne_results = tsne.fit_transform(X)


            with out_input_space:
                        # Plot Results in reduced FS
                        out_input_space.clear_output(wait=True)
                        fig3= plt.figure(figsize=(10, 6))
                        
                        color = [y.sum(axis=1)] # y sind hier die labels
                        sc=plt.scatter(x=tsne_results[:,0], y=tsne_results[:,1], s=100, c=color)
                        palette=sns.color_palette("hls")
                        plt.colorbar(sc)
                        plt.title("Design space in TSNE-coordinates with colored target variable")
                        plt.show()
                        
                        plt.close(fig3)
                        
                        

                
    def fit_DT_wJK(self):        
       
        td= []
        for x in b_resample(self.features_df.iloc[self.SampIdx]):
            td.append(x)
        tl=[]
        for x in b_resample(self.target_df.iloc[self.SampIdx]): 
            tl.append(x)
            
        
        td=np.array(td)
        tl=np.array(tl)
        self.y_pred_dtr=[]
        for i in range(len(td)):
            dtr = DecisionTreeRegressor()
            dtr.fit(td[i], tl[i].ravel())
            self.y_pred_dtr.append(dtr.predict(self.features_df.iloc[self.PredIdx]))
        
       
        #quick bug fix
        if(confirm_strategy()=="MEID (exploit)"):
            self.y_pred_dtr=np.array(self.y_pred_dtr)
            self.Expected_Pred = self.y_pred_dtr.mean(axis=0)
            self.Uncertainty = self.y_pred_dtr.std(axis=0)
        elif(confirm_strategy()=="MLID (explore & exploit)"):
            self.y_pred_dtr=np.array(self.y_pred_dtr)
            self.Expected_Pred = self.y_pred_dtr.mean(axis=0)
            self.Uncertainty = self.y_pred_dtr.std(axis=0)
        else:
            self.y_pred_dtr=np.array(self.y_pred_dtr)
            self.y_pred_dtr=self.y_pred_dtr.T
            self.Expected_Pred = self.y_pred_dtr.mean(axis=1)
            self.Uncertainty = self.y_pred_dtr.std(axis=1)
#        return self.y_pred_dtr,row_means, row_std
        return self.Expected_Pred, self.Uncertainty
        
    def fit_TE_wJK(self):
        
        td= []
        for x in b_resample(self.features_df.iloc[self.SampIdx]):
            td.append(x)
        tl=[]
        for x in b_resample(self.target_df.iloc[self.SampIdx]): 
            tl.append(x)
            
        
        td=np.array(td)
        tl=np.array(tl)
        self.y_pred_dtr=[]
        for i in range(len(td)):
            dtr = BaggingRegressor(n_estimators=10)
            ## alternative Ensamble Learners below:
            #dtr = SKRFR (n_estimators=10)
            #dtr =AdaBoostRegressor(n_estimators = 10)
            dtr.fit(td[i], tl[i].ravel())
            self.y_pred_dtr.append(dtr.predict(self.features_df.iloc[self.PredIdx]))
                
        
        #quick bug fix
        if(confirm_strategy()=="MEID (exploit)"):
            self.y_pred_dtr=np.array(self.y_pred_dtr)
            self.Expected_Pred = self.y_pred_dtr.mean(axis=0)
            self.Uncertainty = self.y_pred_dtr.std(axis=0)
        elif(confirm_strategy()=="MLID (explore & exploit)"):
            self.y_pred_dtr=np.array(self.y_pred_dtr)
            self.Expected_Pred = self.y_pred_dtr.mean(axis=0)
            self.Uncertainty = self.y_pred_dtr.std(axis=0)
        else:
            self.y_pred_dtr=np.array(self.y_pred_dtr)
            self.y_pred_dtr=self.y_pred_dtr.T
            self.Expected_Pred = self.y_pred_dtr.mean(axis=1)
            self.Uncertainty = self.y_pred_dtr.std(axis=1)

#        return self.y_pred_dtr,row_means, row_std
        return self.Expected_Pred, self.Uncertainty        

         
                   
    def fit_RF_wJK(self):

        dtr = RandomForestRegressor()
#        print ('features')
#        print(type(self.features_df.iloc[self.SampIdx]))
#        print(self.features_df.iloc[self.SampIdx])
        dtr.fit(self.features_df.iloc[self.SampIdx].to_numpy(), self.target_df.iloc[self.SampIdx].to_numpy())
        self.Expected_Pred, self.Uncertainty = dtr.predict(self.features_df.iloc[self.PredIdx].to_numpy(), return_std=True)
        return self.Expected_Pred, self.Uncertainty
    
    def fit_GP(self):
        
        kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
        dtr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
        dtr.fit(self.features_df.iloc[self.SampIdx].to_numpy(), self.target_df.iloc[self.SampIdx].to_numpy())
        self.Expected_Pred, self.Uncertainty= dtr.predict(self.features_df.iloc[self.PredIdx], return_std=True)


        """        print('std')
                print(self.Uncertainty.squeeze().shape)
                print('Exp')
                print(self.Expected_Pred.squeeze().shape)

         """       
        return self.Expected_Pred.squeeze(), self.Uncertainty.squeeze()



        
    def main(self):
        self.apply_feature_selection_to_df(self.dataframe)
        self.apply_target_selection_to_df(self.dataframe)
        self.standardize_data()
        self.set_init_sample_size()
        self.set_number_of_ex()
        init_sample_set=self.init_sampling()
        
        
        self.start_sequential_learning()
           
#     def __delete__(self, instance):
#        print ("Deleted in sl object.")
                
        
        


In [450]:
text = widgets.Text(value='Hello World',
            placeholder='',
            description='Customize Strategy:',
            disabled=False)

customize = Button(description='Customize',
                    disabled=False,
                    button_style='warning',
                    tooltip='Click to Confirm Customization',
                    icon='fa-wrench ',
                    layout=widgets.Layout(width='50%'))

def confirm_strategy():
    strategy= select_strategy.value
    if strategy != {}:
        if strategy =='Customize Strategy':
            custom_container.children=[text,customize]
        else:
            custom_container.children=[]
            return select_strategy.value
                
                
                
def add_custom_to_strategy():
    select_strategy.options.append(text.value)

            
        
    

In [451]:
class sl_controller:
    sl_obj=None
    def __init__(self,sl_obj):
        self.sl_obj=sl_obj
        
    def stop(self,sl_obj):
        del sl_obj
        
    def start(self):
        self.sl_obj.main()
        
        
    
        

In [452]:
######EventListener######### 

###Uploads Anfang####


def preview_clicked(b):
    try:
        preview()
    except pd.errors.ParserError:
        with out:
            out.clear_output()
            display(Markdown("Sth. wennt wrong! Please check your csv and the upload settings"))
    
def upload_clicked(b):
    try:
        upload()
    except pd.errors.ParserError:
        with out:
            out.clear_output()
            display(Markdown("Sth. wennt wrong! Please check your csv and the upload settings"))
def desc_clicked(b):
    desc()

    
###Uploads Ende####


###Plot Anfang####
def plotter_clicked(b):
    plot()

def pairwise_clicked(b):
    plot_pairwise()
def heat_clicked(b):
    plot_heat()
def scatter_clicked(b):
    plot_scatter()
    
###Plot Ende####

    
###Selection SL Anfang###

def confirm_features_clicked(b):
    confirm_features()
    
def confirm_target_clicked(b):
    confirm_target()
    
def on_graph_type_change(change):
        
    if graph_type.value =="Scatter":
        container_plot_options.children= [widgets.HBox([select_x,select_y]),
        widgets.HBox([select_hue,select_size])]
    elif graph_type.value =="Scatter Matrix":
        
        container_plot_options.children= [widgets.HBox([selector_plot_variable,button_confirm_plot_var])]
                                          
    elif graph_type.value =='Correlation Heatmap':
        container_plot_options.children= [widgets.HBox([selector_plot_variable,button_confirm_plot_var])]
        
    else: container_plot_options.children=[]


def confirm_var_clicked(b):
    confirm_var()

def confirm_options_clicked(b):
    confirm_options()
    
def customize_clicked(b):
    add_custom_to_strategy()
def confirm_strategy_clicked(b):
    confirm_strategy()
    

def perform_experiment_clicked(b):
    df=df_converter()
    print("type(df)",type(df))
    s = sequential_learning(df)

    s_control= sl_controller(s)
    start_and_stop_sl_container.children=[button_perform_experiment,button_cancel_experiment]
    s_control.start()
    
    
def del_instance(b):
    s_control.stop(s)
    
    
def on_value_change(change):
    quantile_sample_slider.max=quantile_tar_slider.value
def on_strategy_changes(change):
    if select_strategy.value=="MEID (exploit)":
        create_slider_for_dist_quantile()
    elif select_strategy.value=="MLID (explore & exploit)":  
        create_slider_for_dist_quantile()
    else: strategy_container.children=[select_strategy, button_confirm_strategy ]
        
        



In [453]:
#Eventss
#plot

graph_type.observe(on_graph_type_change,names='value')


button_preview.on_click(preview_clicked)


button_confirm_plot_var.on_click(confirm_var_clicked)


quantile_tar_slider.observe(on_value_change,names='value')


#show_results_button.on_click(show_results_button_clicked)

button_confirm_feature.on_click(confirm_features_clicked)
button_confirm_target.on_click(confirm_target_clicked)

#sl
select_strategy.observe(on_strategy_changes,names="value")

button_confirm_options.on_click(confirm_options_clicked)

button_confirm_strategy.on_click(confirm_strategy_clicked)

customize.on_click(customize_clicked)


button_perform_experiment.on_click(perform_experiment_clicked)
button_cancel_experiment.on_click(del_instance)


#upload etc

button_upload.on_click(upload_clicked)
toggle.observe(desc_clicked, 'value')

button_plot.on_click(plotter_clicked)

In [454]:
tab

type(df) <class 'pandas.core.frame.DataFrame'>
Index Max 47 ()
Expected_Pred [[-0.75166658]
 [-0.76483781]
 [-0.76397972]
 [-0.62655517]
 [-0.64108882]
 [-0.6330346 ]
 [-0.46808812]
 [-0.48141985]
 [-0.4002097 ]
 [-0.45964889]
 [-0.3969391 ]
 [-0.40970636]
 [-0.43627905]
 [-0.40254903]
 [-0.72659346]
 [-0.60008058]
 [-0.46788817]
 [-0.4761402 ]
 [-0.37726469]
 [-0.38673478]
 [-0.38080344]
 [-0.26560331]
 [-0.27416616]
 [-0.218343  ]
 [-0.25872284]
 [-0.21570838]
 [-0.2240297 ]
 [-0.24223166]
 [-0.21887087]
 [-0.4757169 ]
 [-0.44989261]
 [-0.35766376]
 [-0.40524782]
 [-0.43376107]
 [-0.3571625 ]
 [-0.34364331]
 [-0.3522592 ]
 [-0.24158755]
 [-0.51869234]
 [-0.51471372]
 [-0.28436502]
 [-0.22633128]
 [-0.33701801]
 [-0.32903846]
 [-0.38846448]
 [-0.38604467]
 [-0.21070653]
 [-0.16746739]
 [-0.24422459]
 [-0.23840059]
 [-1.19267103]
 [-0.66606141]
 [-0.5318781 ]
 [-0.82503554]
 [-0.80696415]
 [-0.89882759]
 [-0.89265924]
 [-0.4936417 ]
 [-0.39371495]
 [-0.60054041]
 [-0.58728554]
 [-0.663

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 131 samples in 0.000s...
[t-SNE] Computed neighbors for 131 samples in 0.003s...
[t-SNE] Computed conditional probabilities for sample 131 / 131
[t-SNE] Mean sigma: 3.034083
[t-SNE] KL divergence after 250 iterations with early exaggeration: 51.136768
[t-SNE] KL divergence after 300 iterations: 0.102739
type(df) <class 'pandas.core.frame.DataFrame'>
Index Max 123 ()
Expected_Pred [[-0.82947076]
 [-0.8561822 ]
 [-0.88255829]
 [-0.79311285]
 [-0.81589425]
 [-0.82824686]
 [-0.73581657]
 [-0.75330641]
 [-0.71247684]
 [-0.75175886]
 [-0.71751425]
 [-0.73404565]
 [-0.75370839]
 [-0.7410831 ]
 [-0.89154694]
 [-0.83249817]
 [-0.58852261]
 [-0.61303419]
 [-0.55406652]
 [-0.57500246]
 [-0.58668464]
 [-0.5020509 ]
 [-0.51791095]
 [-0.4813997 ]
 [-0.51673146]
 [-0.48602692]
 [-0.50107206]
 [-0.51896481]
 [-0.50781376]
 [-0.63819548]
 [-0.64754851]
 [-0.59125041]
 [-0.42778036]
 [-0.48671912]
 [-0.47063208]
 [-0.32056951]
 [-0.24254778]
 [-

/Users/christophvoelker/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:509: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/christophvoelker/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:509: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


Index Max 100 ()
Expected_Pred [[-0.37734665]
 [-0.39592258]
 [-0.40831991]
 [-0.34822854]
 [-0.36350968]
 [-0.36877609]
 [-0.29801386]
 [-0.30981406]
 [-0.27625151]
 [-0.30590069]
 [-0.27859422]
 [-0.28907077]
 [-0.3028874 ]
 [-0.29101423]
 [-0.4046525 ]
 [-0.36492752]
 [-0.20662807]
 [-0.2139867 ]
 [-0.18624986]
 [-0.19256956]
 [-0.19321627]
 [-0.1553653 ]
 [-0.16055274]
 [-0.14197039]
 [-0.15722648]
 [-0.14248265]
 [-0.14706858]
 [-0.15389219]
 [-0.14687101]
 [-0.2170553 ]
 [-0.21114326]
 [-0.18829962]
 [-0.30819001]
 [-0.29432594]
 [-0.21063123]
 [-0.31233817]
 [-0.24193546]
 [-1.04509913]
 [-0.72482041]
 [-0.60932562]
 [-0.90129167]
 [-0.87804348]
 [-0.85715232]
 [-0.82609426]
 [-0.69707096]
 [-0.6086533 ]
 [-0.72751478]
 [-0.70393637]
 [-0.63616899]
 [-0.5930254 ]
 [-0.82124198]
 [-0.77341139]
 [-0.54097667]
 [-0.51332395]
 [-0.73963506]
 [-0.67437301]
 [-1.05632718]
 [-0.64411607]
 [-0.60452996]
 [-0.28326426]
 [-0.27684882]
 [-0.26798873]
 [-0.25699118]
 [-0.24450539]
 [-0.2796

/Users/christophvoelker/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:509: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/christophvoelker/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:509: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/christophvoelker/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:509: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale 

...

Index Max 110 ()
Expected_Pred [[-0.45250216]
 [-0.46775218]
 [-0.48309611]
 [-0.36382749]
 [-0.37978285]
 [-0.38200439]
 [-0.25406885]
 [-0.26739362]
 [-0.20731941]
 [-0.25505092]
 [-0.20713475]
 [-0.24411038]
 [-0.22076098]
 [-0.47479359]
 [-0.36980642]
 [-0.86700912]
 [-0.77324272]
 [-0.78663227]
 [-0.78646381]
 [-0.66822726]
 [-0.68022512]
 [-0.62186201]
 [-0.66738203]
 [-0.62114568]
 [-0.63344965]
 [-0.65496042]
 [-0.63175826]
 [-0.87655569]
 [-0.86388186]
 [-0.7718106 ]
 [-0.94679731]
 [-0.90849305]
 [-0.81235317]
 [-0.91738614]
 [-0.93138895]
 [-0.82372276]
 [-1.02224291]
 [-1.01135248]
 [-1.16069843]
 [-1.16845521]
 [-0.81964305]
 [-0.80601392]
 [-1.07411381]
 [-1.18051708]
 [-1.18143002]
 [-0.88331081]
 [-0.87077533]
 [-0.82492693]
 [-0.80824695]
 [-1.07948055]
 [-0.57969992]
 [-0.56196625]
 [-0.86288649]
 [-0.84805365]
 [-1.08067077]
 [-1.10566409]
 [-0.63389811]
 [-0.61758996]
 [-0.73085523]
 [-0.70076055]
 [-0.66901566]
 [-0.63596366]
 [-0.60259463]
 [-0.71900485]
 [-0.6793

Index Max 126 ()
Expected_Pred [[-0.5096362 ]
 [-0.51163169]
 [-0.50502879]
 [-0.38508272]
 [-0.39294058]
 [-0.38287283]
 [-0.23877991]
 [-0.24820453]
 [-0.17703155]
 [-0.22784526]
 [-0.17318675]
 [-0.18348869]
 [-0.17707829]
 [-0.46996939]
 [-0.35242786]
 [-1.23274282]
 [-1.25446106]
 [-1.19761279]
 [-1.21816796]
 [-1.22160859]
 [-1.11940737]
 [-1.13982922]
 [-1.08129243]
 [-1.13320902]
 [-1.10470465]
 [-1.1270423 ]
 [-1.25750828]
 [-1.23902893]
 [-1.21026399]
 [-1.17317096]
 [-1.25854026]
 [-1.05413105]
 [-0.85883022]
 [-0.86685611]
 [-0.77662034]
 [-0.63384576]
 [-0.64087653]
 [-0.37855081]
 [-0.31776046]
 [-0.49565147]
 [-0.49104434]
 [-0.49583598]
 [-0.50133591]
 [-0.29612742]
 [-0.2485732 ]
 [-0.38773129]
 [-0.38412729]
 [-1.18176016]
 [-0.68162642]
 [-0.57064666]
 [-0.8590071 ]
 [-0.85299552]
 [-0.92445078]
 [-0.94050197]
 [-0.53321317]
 [-0.44639748]
 [-0.67197204]
 [-0.66726939]
 [-0.69749687]
 [-0.68096727]
 [-0.66010594]
 [-0.63577093]
 [-0.60911573]
 [-0.71783743]
 [-0.6898

Index Max 122 ()
Expected_Pred [[-1.00711403]
 [-1.00735234]
 [-0.98951432]
 [-0.99284035]
 [-0.99408427]
 [-0.98543393]
 [-0.94617227]
 [-0.95149283]
 [-0.91794565]
 [-0.94223461]
 [-0.91680316]
 [-0.92122433]
 [-0.92933449]
 [-0.91475787]
 [-0.96207742]
 [-0.96626545]
 [-0.55287592]
 [-0.55189162]
 [-0.54108005]
 [-0.54104324]
 [-0.53533404]
 [-0.51180443]
 [-0.51427243]
 [-0.49477913]
 [-0.50838699]
 [-0.49373338]
 [-0.49580083]
 [-0.50031709]
 [-0.49165527]
 [-0.54083795]
 [-0.52408654]
 [-0.52338468]
 [-0.34857968]
 [-0.46700834]
 [-0.46300341]
 [-0.11585715]
 [-0.09654376]
 [-0.35357446]
 [-0.35774658]
 [-0.21863346]
 [-0.18542745]
 [-0.34924309]
 [-0.34971178]
 [-0.37349057]
 [-0.37855863]
 [-0.2280451 ]
 [-0.19309854]
 [-0.35785292]
 [-0.35834818]
 [-1.16357416]
 [-1.16904909]
 [-0.7428958 ]
 [-0.6306653 ]
 [-1.20072435]
 [-1.11898426]
 [-0.70422023]
 [-0.59706394]
 [-1.12333159]
 [-1.12235153]
 [-0.32115525]
 [-0.32186347]
 [-0.32044839]
 [-0.31711786]
 [-0.33547315]
 [-0.3333

Index Max 122 ()
Expected_Pred [[-0.59842077]
 [-0.59956012]
 [-0.56569032]
 [-0.59050505]
 [-0.59346699]
 [-0.57958465]
 [-0.52855568]
 [-0.53863119]
 [-0.49041442]
 [-0.52666193]
 [-0.49030865]
 [-0.49803856]
 [-0.50930444]
 [-0.48962932]
 [-0.51936639]
 [-0.54970416]
 [-0.18585081]
 [-0.18616741]
 [-0.18306141]
 [-0.1839006 ]
 [-0.17947001]
 [-0.16363372]
 [-0.16668013]
 [-0.15174313]
 [-0.16289655]
 [-0.15166384]
 [-0.15397619]
 [-0.15740322]
 [-0.15127982]
 [-0.17556266]
 [-0.16094755]
 [-0.16999853]
 [-0.21199789]
 [-0.25944672]
 [-0.13133225]
 [-0.11233009]
 [-0.09761936]
 [-0.08613362]
 [-0.08777591]
 [-0.01899802]
 [-0.01134301]
 [-0.10262128]
 [-0.10208934]
 [-0.09729181]
 [-0.10063306]
 [-0.02047285]
 [-0.01206771]
 [-0.08851823]
 [-0.08819945]
 [-1.01707554]
 [-1.02231308]
 [-0.25564379]
 [-0.15249583]
 [-1.18946268]
 [-1.09515131]
 [-0.24248322]
 [-0.14302047]
 [-1.00089943]
 [-0.99482   ]
 [-0.27302886]
 [-0.29085652]
 [-0.30358108]
 [-0.31071614]
 [-0.29964876]
 [-0.3143

AttributeError: 'float' object has no attribute 'squeeze'